In [1]:
# Import packages
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')
from sklearn.ensemble import RandomForestRegressor
from IPython.display import clear_output

# Set working path
path = '/Users/martinbogaert/Desktop/NBA Data Analysis/nba_ai_awards/'

print('MAKE SURE YOU CHANGE THE WEEK VARIABLE')

MAKE SURE YOU CHANGE THE WEEK VARIABLE


In [2]:
week = 19 # CHANGE THIS
date = datetime.now().strftime('%d %B %Y')
time = datetime.now().strftime('%H:%M:%S')
print(date + ' ; ' + time)

27 February 2023 ; 12:58:15


In [3]:
features = ['PTS','VORP','FG','WS','MP','TRB','AST','PER','BPM']#, 'GS'] ### GS
model = RandomForestRegressor(n_estimators = 100, max_features = 'auto', min_samples_leaf = 2)

# Minimum minutes per game
mp = 8
# Propotion of maximum games played
gp = 1/4

In [4]:
# Load training data
train_data = pd.read_csv(path + 'Algorithm/roy/roy_data.csv')

# Load predicted data
data = pd.read_csv(path + f'Algorithm/weekly data/week_{week}.csv') # Load up-to-date data
data = data.groupby('roy').get_group(1)

# Load weekly info
with open(path + f'Algorithm/weekly data/week_{week}.txt', 'r') as file:
            date1 = file.readline().splitlines()[0][7:]
            date2 = file.readline().splitlines()[0][7:]
            n = int(file.readline()[12:])
            
season_progress = n / 15
data.head()

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,MOV,ORtg,DRtg,NRtg,MOV/A,ORtg/A,DRtg/A,NRtg/A,roy,smoy
0,A.J. Green,SG,23,MIL,27,0,9.7,1.6,3.6,0.449,...,3.65,114.60,111.05,3.55,3.57,114.68,111.19,3.49,1,1
10,MarJon Beauchamp,SF,22,MIL,39,9,14.6,2.1,5.2,0.396,...,3.65,114.60,111.05,3.55,3.57,114.68,111.19,3.49,1,1
17,A.J. Lawson,SG,22,DAL,13,0,4.8,1.0,2.0,0.500,...,0.84,117.54,116.70,0.84,0.74,117.32,116.60,0.72,1,1
25,Jaden Hardy,SG,20,DAL,32,0,11.6,2.2,5.1,0.423,...,0.84,117.54,116.70,0.84,0.74,117.32,116.60,0.72,1,1
38,AJ Griffin,SF,19,ATL,57,11,20.2,3.6,7.6,0.475,...,-0.03,115.99,116.02,-0.02,-0.09,116.18,116.26,-0.08,1,1


In [5]:
# Set training data
X_train = train_data[features]
y_train = train_data['Share']

# Set minimum requirements (games and minutes played)
data = data[data['MP'] >= mp] # Minimum of minutes per game
data = data[data['G'] > season_progress * gp] # Minimum games played

pred = []
for step in range(0, 10) :
    print('Model ' + str(step+1) + '/10 ...')
    clear_output(wait = True)
    # Fit Machine Learning model
    model.fit(X_train, y_train)

    # Predict shares of test data
    pred.append(model.predict(data[features]))

Model 10/10 ...


In [6]:
# Assemble results DataFrame
res = data.assign(Share = [np.array(pred)[:,i].mean() for i in range(0, len(data))])
res = res.sort_values('Share', ascending = False) # Sort values by predictions
res['Rank'] = list(range(1, len(res)+1))
res['week'] = len(res) * [week] # Add week

res = res.reset_index(drop = True)[['Player', 'MP' ,'Tm', 'PER', 'VORP', 'WS', 'FG', 'PTS', 'Seed', 'Share']]
res.head(5)

,Player,MP,Tm,PER,VORP,WS,FG,PTS,Seed,Share
0,Paolo Banchero,33.7,ORL,14.9,0.303704,2.733333,6.4,19.7,13,0.239152
1,Bennedict Mathurin,28.3,IND,13.8,-0.661290,2.380645,5.5,17.3,12,0.147939
2,Walker Kessler,21.6,UTA,21.1,2.084746,7.227119,3.6,8.2,8,0.117159
3,Jaden Ivey,30.1,DET,12.3,-0.575439,0.431579,5.2,15.3,15,0.094763
4,Jalen Duren,25.1,DET,16.5,0.473077,5.519231,3.6,8.5,15,0.093390


In [7]:
res.to_csv(path + f'Results/roy/results_week_{week}.csv', index = None)